# Pipeline Concepts

Some key terms:
- `Pipeline`: encapsulates your entire data processing task from start to finish (read in, transform, write out). All Beam river programs must create a Pipeline.
- `PCollection`: represents a distributed data set that the Beam pipeline operates on. It can either be bounded (fixed source/flat file) or unbounded (data stream). Reading in input at the start of the `Pipeline` generally creates a `PCollection`, but you can also create one from in-memory data. `PCollections` are the inputs and outputs for each step in your pipeline.
- `PTransform`: a data processing operation or step in your `Pipeline`. Every `PTransform` takes one or more `PCollection` objects as an input, then returns 0 or more `PCollection`s.
    - This can be any change, filter, group, analyze process. Each `PTransform` creates a new output without modifying the input collection.
- `I/O transforms`: Beam comes with various "IOs", `PTransforms` that read or write data to various systems.

Generally, a Beam pipeline will:
- Create a `Pipeline` object and set pipeline execution options (including the *Pipeline Runner* - Direct (in-mem) or Dataflow (Google Dataflow))
- Create an initial `PCollection` using IOs or from in-memory data
- Apply `PTransforms` to each `PCollection` wher necessary
- Use IOs to write the final `PCollection` to an external source
- Run the `Pipeline` using the designated *Pipeline Runner*

Some more Beam terms:
- `Aggregation`: computing a value from multiple input elements
- `User-defined function (UDF)`: some Beam operations allow you to run user-defined code as a way to configure a transform
- `Schema`: language-independent type definition for a `PCollection`
- `SDK`: language-specific library that lets pipeline authors build transforms, construct pipelines, and submit them to a runner
- `Window`: A `PCollection` can be subdivided into windows based on the timestamps of the individual elements. Windows enable grouping operations overcollections that grow over time
- `Watermark`: a guess as to when all data in a certain window is expected to have arrived
- `Trigger`: determins when to aggregate the results of each window

## Creating pipelines

The `Pipeline` encapsulates all the data and steps in your data pocessing task. You start by constructing a `Pipeline` object and using that object as the basis for the pipeline's operations.

In [1]:
import apache_beam as beam

with beam.Pipeline() as p:
    pass

You can pass explicitly specified parameters this way:


In [6]:
from apache_beam.options.pipeline_options import PipelineOptions

beam_options = PipelineOptions(
    flags  = [], # This must be here otherwise it will pull in sys.argv https://stackoverflow.com/a/75265512
    runner = 'DirectRunner' # or 'DataflowRunner
    # project       = 'my-proj',         # GCP Dataflow option
    # job_name      = 'unique-job-name', # GCP Dataflow option
    # temp_location = 'gs://my-bucket',  # GCP Dataflow option
)

with beam.Pipeline(options=beam_options) as p:
    pass

## Configuring pipeline options

You can read options from the command line by simply setting up `PipelineOptions` (can be empty), to have it take in command line args, do not define the `flags` argument. This will interpret command-line arguments as: 

```shell
--<option>=<value>
```

You can also create custom options by creating a `PipelineOptions` subclass:

```python
class MyOptions(PipelineOptions):
    @classmethod
    def _add_argparse_args(cls, parser):
        parser.add_argument(
            '--input'
            default='gs://dataflow-samples/shakespear/kinglear.txt',
            help='The file path for the input text to process'
        )
        parser.add_argument(
            '--output',
            required=True
        )
```

Or by simply parsing the custom options with `argparse`:

```python
import argparse

from apache_beam.io import ReadFromText
from apache_beam.io import WriteToText
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import SetupOptions

def main(argv=None, save_main_session=True):
    
    # Define new arguments
    parser = argparse.ArgumentParser()
    parser.add_argument(
        '--input',
        dest='input',
        default='gs://dataflow-samples/shakespear/kinglear.txt',
        help='The file path forthe input text to process'
    )
    parser.add_argument(
        '--output',
        dest='output',
        required=True
    )

    # Split arguments into the args defined above, and all other args 
    # (which we assume go directly into `PipelineOptions`)
    known_args, pipeline_args = parser.parse_known_args(argv)

    # Run pipeline args through PipelineOptions
    pipeline_options = PipelineOptions(pipeline_args)

    # Use save_main_session if our workflow relies on the global context 
    # (e.g. a module defined at the global level)
    pipeline_options.view_as(SetupOptions).save_main_session = save_main_session

    with beam.Pipeline(options=pipeline_options) as p:

        lines = (
            p
            # Using an additional arg directly in pipeline
            | 'Read' >> ReadFromText(known_args.input)
            | beam.Filter(lambda line: line != "")
        )

        output = lines | 'Write' >> WriteToText(known_args.output)

        result = p.run()
        result.wait_until_finish()

```

In [12]:
%%bash

python source/02-pipeline-concepts.py --help # argparse automatically builds help docco when adding args

usage: 02-pipeline-concepts.py [-h] [--input INPUT] [--output OUTPUT]

optional arguments:
  -h, --help       show this help message and exit
  --input INPUT    The file path for the input text to process
  --output OUTPUT


In [13]:
%%bash

python source/02-pipeline-concepts.py \
    --output "output/agatha-count"